In [1]:
%cd /Users/shivamkaushik/Code/ik-agent

/Users/shivamkaushik/Code/ik-agent


In [2]:
from src.agent import agent

from IPython.display import display, Markdown

def display_agent_message(agent_message: dict):
    content = agent_message.get("content", [])
    text_blocks = [
        block["text"] for block in content if block.get("text")
    ]
    markdown = "\n\n".join(text_blocks)
    print('----------------------------\n\n')
    display(Markdown(markdown))

Deprecation warning: AnalyticsRulesV1 is deprecated on v30+. Use client.analytics instead.
/Users/shivamkaushik/Code/ik-agent/.venv/lib/python3.12/site-packages/pydantic/main.py:1809: UserWarning: Field name "schema" in "Create_custom_metadata_fields_toolTool" shadows an attribute in parent "BaseModel"
  return meta(
/Users/shivamkaushik/Code/ik-agent/.venv/lib/python3.12/site-packages/pydantic/main.py:1809: UserWarning: Field name "schema" in "Update_custom_metadata_fields_toolTool" shadows an attribute in parent "BaseModel"
  return meta(


In [3]:
search_query = 'name="01.JPG"'
body = {
    "search_query": search_query,
}

In [4]:
from src.clients import CLIENT
raw_assets = await CLIENT.assets.list(**body)

INFO | httpx | HTTP Request: GET https://api.imagekit.io/v1/files?searchQuery=name%3D%2201.JPG%22 "HTTP/1.1 200 OK"


In [5]:
from typing import Any

def _serialize_asset(asset: Any) -> Any:
    """
    Normalize SDK asset objects into plain dicts for filtering/returning.
    """
    if hasattr(asset, "model_dump"):
        return asset.model_dump()
    if hasattr(asset, "dict"):
        return asset.dict()
    return asset
response = [_serialize_asset(asset) for asset in raw_assets]

In [6]:
from src.utils.utils import maybe_filter
# filter_spec = 'file_id'
filter_spec = '[0].file_id'
maybe_filter(filter_spec, response)

INFO | utils.utils | ('There was some problem accessing the response fields you requested. ', 'Requested [0].file_id', "Got response: [{'ai_tags': [{'confidence': 98.8, 'name': 'Photograph', 'source': 'google-auto-tagging'}, {'confidence': 97.42, 'name': 'Night', 'source': 'google-auto-tagging'}, {'confidence': 95.46, 'name': 'Tower', 'source': 'google-auto-tagging'}, {'confidence': 93.8, 'name': 'Landmark', 'source': 'google-auto-tagging'}, {'confidence': 91.71, 'name': 'Lighting', 'source': 'google-auto-tagging'}, {'confidence': 89.19, 'name': 'Spire', 'source': 'google-auto-tagging'}, {'confidence': 89.07, 'name': 'Monument', 'source': 'google-auto-tagging'}], 'created_at': datetime.datetime(2026, 1, 12, 7, 24, 45, 727000, tzinfo=TzInfo(0)), 'custom_coordinates': None, 'custom_metadata': {}, 'description': None, 'file_id': '6964a1bd5c7cd75eb89c492b', 'file_path': '/ik_agent_images/easy_usecases/01.JPG', 'file_type': 'image', 'has_alpha': False, 'height': 640.0, 'is_private_file': Fa

[{'ai_tags': [{'confidence': 98.8,
    'name': 'Photograph',
    'source': 'google-auto-tagging'},
   {'confidence': 97.42, 'name': 'Night', 'source': 'google-auto-tagging'},
   {'confidence': 95.46, 'name': 'Tower', 'source': 'google-auto-tagging'},
   {'confidence': 93.8, 'name': 'Landmark', 'source': 'google-auto-tagging'},
   {'confidence': 91.71, 'name': 'Lighting', 'source': 'google-auto-tagging'},
   {'confidence': 89.19, 'name': 'Spire', 'source': 'google-auto-tagging'},
   {'confidence': 89.07, 'name': 'Monument', 'source': 'google-auto-tagging'}],
  'created_at': datetime.datetime(2026, 1, 12, 7, 24, 45, 727000, tzinfo=TzInfo(0)),
  'custom_coordinates': None,
  'custom_metadata': {},
  'description': None,
  'file_id': '6964a1bd5c7cd75eb89c492b',
  'file_path': '/ik_agent_images/easy_usecases/01.JPG',
  'file_type': 'image',
  'has_alpha': False,
  'height': 640.0,
  'is_private_file': False,
  'is_published': True,
  'mime': 'image/jpeg',
  'name': '01.JPG',
  'selected_fie

In [7]:
from typing import List, Dict, Any

response: List[Dict[str, Any]] = [
    {'file_id': '1627736', 'file_name': 'abc.jpg'},
    {'file_id': '326763', 'file_name': 'cde.jpg'},
]

spec = [{'file_id': 'file_id'}]
maybe_filter(spec, response)

[{'file_id': '1627736'}, {'file_id': '326763'}]

In [8]:
response: Dict[str, Any] = {'file_id': '1627736', 'file_name': 'abc.jpg'}

spec = 'file_id'
maybe_filter(spec, response)

'1627736'

In [9]:
response: Dict[str, Any] = {'file_id': '1627736', 'file_name': 'abc.jpg'}

spec = {'file_id': 'file_id', 'file_name': 'file_name'}
maybe_filter(spec, response)

{'file_id': '1627736', 'file_name': 'abc.jpg'}

In [10]:
response: Dict[str, Any] = {'file': {
    'id': '1627736',
    'name': 'abc.jpg',
    'metadata': {
        'size': 2048,
        'type': 'image/jpeg'
    }
}}

spec = {'file_metadata_size': 'file.metadata.size'}
maybe_filter(spec, response)

{'file_metadata_size': 2048}

In [12]:
import re

def jq_to_glom(jq_filter: str):
    """
    Convert a limited subset of jq filter syntax into a glom spec.

    Supported:
    - .a.b.c
    - .a[].b
    - .a[0].b
    - .a[]
    - .a | .b
    - .a, .b
    """

    jq_filter = jq_filter.strip()

    # Handle comma operator (multiple outputs)
    if ',' in jq_filter:
        parts = [p.strip() for p in jq_filter.split(',')]
        return tuple(jq_to_glom(p) for p in parts)

    # Handle pipe operator
    if '|' in jq_filter:
        parts = [p.strip() for p in jq_filter.split('|')]
        combined = []
        for p in parts:
            spec = jq_to_glom(p)
            if isinstance(spec, str):
                combined.extend(spec.split('.'))
            else:
                combined.append(spec)
        return '.'.join(str(x) for x in combined)

    # Remove leading dot
    if jq_filter.startswith('.'):
        jq_filter = jq_filter[1:]

    tokens = re.split(r'\.(?![^\[]*\])', jq_filter)

    glom_spec = []

    for token in tokens:
        # Match array access
        array_match = re.match(r'(\w+)\[(.*?)\]$', token)

        if array_match:
            key, idx = array_match.groups()
            glom_spec.append(key)

            if idx == '':
                # []
                glom_spec.append([])
            else:
                # [0]
                glom_spec.append(int(idx))
        else:
            glom_spec.append(token)

    # Simplify output
    if len(glom_spec) == 1:
        return glom_spec[0]

    # Convert to glom tuple form
    return tuple(glom_spec)

spec = jq_to_glom('[0].file_id')
response: List[Dict[str, Any]] = [
    {'file_id': '1627736', 'file_name': 'abc.jpg'},
    {'file_id': '326763', 'file_name': 'cde.jpg'},
]

maybe_filter(spec, response)

INFO | utils.utils | ('There was some problem accessing the response fields you requested. ', "Requested ('[0]', 'file_id')", "Got response: [{'file_id': '1627736', 'file_name': 'abc.jpg'}, {'file_id': '326763', 'file_name': 'cde.jpg'}]")


[{'file_id': '1627736', 'file_name': 'abc.jpg'},
 {'file_id': '326763', 'file_name': 'cde.jpg'}]

In [16]:
import json

json.loads('file_id')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)